# Worksheet 05

Name:  Thian Amarasekera

UID: U91933393

### Topics

- Cost Functions
- Kmeans

### Cost Function

Solving Data Science problems often starts by defining a metric with which to evaluate solutions were you able to find some. This metric is called a cost function. Data Science then backtracks and tries to find a process / algorithm to find solutions that can optimize for that cost function.

For example suppose you are asked to cluster three points A, B, C into two non-empty clusters. If someone gave you the solution `{A, B}, {C}`, how would you evaluate that this is a good solution?

Notice that because the clusters need to be non-empty and all points must be assigned to a cluster, it must be that two of the three points will be together in one cluster and the third will be alone in the other cluster.

In the above solution, if A and B are closer than A and C, and B and C, then this is a good solution. The smaller the distance between the two points in the same cluster (here A and B), the better the solution. So we can define our cost function to be that distance (between A and B here)!

The algorithm / process would involve clustering together the two closest points and put the third in its own cluster. This process optimizes for that cost function because no other pair of points could have a lower distance (although it could equal it).

### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

1. assign each point in the dataset to its cloest center [0, .5], [1.5, 2, 6, 6.5, 7] 
2. recompute the center of each cluster [0.25, 4.6] 
3. assign each point in the dataset to its cloest center [0, .5, 1.5, 2], [6, 6.5, 7] 
4. recompute the center of each cluster [1.0, 6.5] 
5. assign each point in the dataset to its cloest center [0, .5, 1.5, 2], [6, 6.5, 7] 
6. convergence

b) Describe in plain english what the cost function for k means is.

The cost function for k is the sum of the squared distance between each point and its closest center.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

This is because the initial centroids are randomly chosen. The algorithm will converge to a local minimum, which depends on the initial centroids.

d) Does Lloyd's Algorithm always converge? Why / why not?

Yes, because the cost function is monotonically decreasing, and the number of possible assignments is finite so that means the algorithm will converge.

e) Follow along in class the implementation of Kmeans

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []
    
    def distance(self, x, y):
        return np.linalg.norm(x - y)
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def initialize(self):
        return self.data[np.random.choice(len(self.data), self.k, replace=False)]

    def assign(self, centers):
        for i in range(len(self.data)):
            min = self.distance(self.data[i], centers[0])
            self.assignment[i] = 0
            # self.assignment = 0
            for j in range(len(centers)):
                dist = self.distance(self.data[i], centers[j])
                if dist < min:
                    min = dist
                    self.assignment[i] = j

        # for i, x in enumerate(self.data):
        #     self.assignment[i] = np.argmin([np.linalg.norm(x - c) for c in centers])

    def is_diff_clusters(self, centers, new_centers):
        for i in range(len(centers)):
            if self.distance(centers[i], new_centers[i]) != 0:
                return True
        return False

    def get_centers(self):
        new_centers = []
        for i in set(self.assignment): # for every different assignment
            cluster = self.data[np.array(self.assignment) == i]
            new_centers.append(np.mean(cluster, axis=0))

        return np.array(new_centers)
    
    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.get_centers()

        while self.is_diff_clusters(centers, new_centers):
            self.assign(new_centers)
            centers = new_centers
            self.snap(new_centers)
            new_centers = self.get_centers()
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps # takes a snapshot of each stage of the algorithm

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)